# How would Height of the basketball player affect the force it takes to make a free throw?

In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

Fm = S (w × v)

In [2]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
degree = UNITS.degree
rad = UNITS.radian

<Unit('radian')>

In [3]:

params = Params(x = 0 * m,                   # x coordinate of the basketball
                y = 0 * m,                   # y coordinate of the basketball
                omega = 0 * rad / s ** 2,    # velocity
                rho = 1.2 * kg / m ** 3,     # mass density of the fluid
                C_d = 0.54,                  # drag coefficient of a basketball
                x_dist = 4.9 * m,            # horizontal distance of player to net
                y_dist = 3.05 * m,           # distance of the rim from ground
                v = 0 * m / s,               # velocity of the basketball
                S = 0.47,                    # air resistance coefficient of spheres
                g = 9.8 * m/s**2,            # gravity
                angle = 52 * degree,         # launch angle of the basketball
                mass = 0.625 * kg,           # mass of the basketball
                diameter = 0.23 * m,        # diameter of the basketball
                t_end = 10 * s)

,values
x,0 meter
y,0 meter
omega,0.0 radian / second ** 2
rho,1.2 kilogram / meter ** 3
C_d,0.54
x_dist,4.9 meter
y_dist,3.05 meter
v,0.0 meter / second
S,0.47
g,9.8 meter / second ** 2


In [4]:
def make_system(params, v_0, height):
    """Make a system object.
    
    params: 
    
    returns: System object
    """
    unpack(params)
     # convert angle to degrees
    theta = np.deg2rad(angle)
    
    # compute x and y components of velocity
    vx, vy = pol2cart(theta, v_0)

    init = State(x = x, y = height, vx = vx, vy = vy)

    # compute area from diameter
    area = np.pi * (diameter/2)**2
    
    return System(params, init=init, area = area)

In [5]:
system = make_system(params, 0 * m / s, 1.8 * m)

,values
x,0 meter
y,0 meter
omega,0.0 radian / second ** 2
rho,1.2 kilogram / meter ** 3
C_d,0.54
x_dist,4.9 meter
y_dist,3.05 meter
v,0.0 meter / second
S,0.47
g,9.8 meter / second ** 2


In [6]:
def drag_force(V, system):
    """Computes drag force in the opposite direction of `V`.
    
    V: velocity
    system: System object with rho, C_d, area
    
    returns: Vector drag force
    """
    unpack(system)
    mag = -rho * V.mag**2 * C_d * area / 2
    direction = V.hat()
    f_drag = mag * direction
    return f_drag

In [7]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, x velocity, y velocity)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (vx, vy, ax, ay)
    """
    x, y, vx, vy = state
    unpack(system)

    V = Vector(vx, vy)    
    a_drag = drag_force(V, system) / mass
    a_grav = Vector(0, -g)
    
    a = a_grav + a_drag
    
    return vx, vy, a.x, a.y

In [8]:
def event_func(state, t, system):
    """Stop when the y coordinate is 0.
    
    state: State object
    t: time
    system: System object
    
    returns: y coordinate
    """
    x, y, vx, vy = state
    return x - 4.9

In [9]:
def height_func(v_0, height):
    system = make_system(params, v_0 = v_0, height = height)
    results, details = run_ode_solver(system, slope_func, events=event_func)
    return Vector(get_last_value(results.x) * m, get_last_value(results.y) * m)

In [10]:
height_func(20 * m /s, 1.8*  m)

<Quantity([4.9        7.20318324], 'meter')>

In [32]:
velocities = linspace(6, 12, 41)
heights = linspace(1.6, 2.3, 8)
sweep_h = SweepSeries()

print(heights)
for h in heights:
    sweep_v = SweepSeries()
    for vel in velocities:
        vector = height_func(vel * m / s, h * m)
        if(vector.y.mag < 3.165 * m and vector.y.mag > 2.935 * m):
            print(h, vel, vector)
            sweep_v[vel] = vector
    sweep_h[h] = sweep_v.index[0]
sweep_h

[1.6 1.7 1.8 1.9 2.  2.1 2.2 2.3]
1.6 8.4 [4.9        3.03277826] meter
1.7 8.25 [4.9        2.95734244] meter
1.7 8.4 [4.9        3.13276652] meter
1.8 8.25 [4.9       3.0573234] meter
1.9 8.1 [4.9        2.97205956] meter
1.9 8.25 [4.9        3.15730539] meter
2.0 8.1 [4.9        3.07203417] meter
2.1 7.95 [4.9        2.97618618] meter
2.1999999999999997 7.95 [4.9        3.07615472] meter
2.3 7.8 [4.9        2.96889123] meter


,values
1.6,8.40
1.7,8.25
1.8,8.25
1.9,8.10
2.0,8.10
2.1,7.95
2.2,7.95
2.3,7.80


In [ ]:
sweep_h.index